# AutoGen-työkalun käyttöesimerkki


## Tuo tarvittavat paketit


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Asiakkaan luominen

Tässä esimerkissä käytämme [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) -palvelua LLM:n käyttöön.

`model` on määritelty nimellä `gpt-4o-mini`. Kokeile vaihtaa mallia johonkin toiseen GitHub Models -markkinapaikalla saatavilla olevaan malliin nähdäksesi erilaisia tuloksia.

Nopeana testinä suoritetaan yksinkertainen kysely - `Mikä on Ranskan pääkaupunki`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Funktioiden määrittäminen

Tässä esimerkissä annamme agentille pääsyn työkaluun, joka on funktio, jossa on luettelo saatavilla olevista lomakohteista ja niiden saatavuudesta.

Voit ajatella, että tämä voisi olla tilanne, jossa matkatoimistolla on esimerkiksi pääsy matkailutietokantaan.

Kun käyt läpi tätä esimerkkiä, voit vapaasti kokeilla määritellä uusia funktioita ja työkaluja, joita agentti voi käyttää.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Määritellään Function Tool 
Jotta agentti voi käyttää `vacation_destinations`-työkalua `FunctionTool`-ominaisuutena, meidän täytyy määritellä se sellaiseksi.

Annamme myös työkalulle kuvauksen, joka auttaa agenttia ymmärtämään, mihin kyseistä työkalua käytetään suhteessa käyttäjän pyytämään tehtävään.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Agentin määrittäminen

Nyt voimme luoda agentin alla olevassa koodissa. Määritämme `system_message`-viestin, joka antaa agentille ohjeet siitä, miten auttaa käyttäjiä löytämään lomakohteita.

Asetamme myös `reflect_on_tool_use`-parametrin arvoon true. Tämä mahdollistaa LLM:n käyttämään työkalukutsun vastausta ja lähettämään vastauksen luonnollisella kielellä.

Voit asettaa parametrin arvoon false nähdäksesi eron.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Agentin käynnistäminen

Nyt voimme käynnistää agentin käyttäjän alkuperäisellä viestillä, jossa pyydetään matkaa Tokioon.

Voit vaihtaa tämän kaupungin kohteen nähdäksesi, miten agentti reagoi kaupungin saatavuuteen.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
